In [1]:
from KeRLas import Brain, TimedGymEnv, Trainer
from KeRLas.models import defaultQModel
from KeRLas.models import DirectDiffModel, NaiveQModel, LateralDiffModel, DQN
from KeRLas.policies import BoltzmannQPolicy
import numpy as np
import gym, math
from cartpole import CartPoleEnv

#env = GymEnv(CartPoleEnv(), tlimit=300)
env = TimedGymEnv(CartPoleEnv(), tlimit=300)

space = env.observation_space
high = np.array(
    [
        2.4,
        1.0,
        12 * 2 * math.pi / 360,
        1.0
    ]
)
env.RandomObservationSpace = gym.spaces.Box(-high, high)

observation_width = env.observation_space.shape[0]
nactions = env.action_space.n


#for i in range(10):
#    next(brain.Memory.generate_samples(20))
#_ = next(brain.trainig_data_generator(mbsize))



Using TensorFlow backend.


WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.


In [2]:
rlmodel = DQN(defaultQModel(observation_width, nactions), 0.9, soft_update=0.1)
policy = BoltzmannQPolicy(0.0001)
brain = Brain(rlmodel, policy, training_policies=map(BoltzmannQPolicy, [10.0, 1.0, 0.1, 0.01, 0.001]))


defaultQModel: 6
6 6


In [3]:
class Callback:
    
    def onEpisodeBegin(self, env, agents, observations):
        env.render()

    def onStep(self, env, env_done, feedback):
        env.render()
        
    def onEpisodeEnd(self, env, record):
        env.render()

trainer = Trainer(env, brain, 0.00, 10000)   

from KeRLas import GymPlayer
p = GymPlayer(env, brain, callback=Callback())

#temps = [10.0, 1.0, 0.1, 0.01, 0.001]

mbsize = 50

for i in xrange(20):
    metrics = trainer.train(mbsize, 10000, 10)
    
    nsum = 0
    N = 5
    with brain.training(False):
        for _ in range(N):
            history = p.runEpisode()
            #print len(history)
            nsum += len(history)
        print float(nsum)/N, metrics

45.8 1059.1603
38.6 7458.489
29.4 32.638844
28.6 45.827816
32.6 29.323118
26.8 11.909241
24.0 3.3099093
29.0 9.792427
43.8 2.377565
12.8 0.11507924
75.8 2.689246
53.8 59098.574
32.6 161.88223
40.8 15.340041
36.4 3.0495572
34.2 4.462639
33.8 1.7901182
32.4 7.0952606
29.8 1.7194766
31.4 1.2098132
